In [1]:
import sys
sys.path.insert(0, '/root/mounted/projects/qpair/')

In [2]:
from config import *

In [3]:
# 加载模块
import pandas as pd
import pickle
import os
import numpy as np
from add_vectors_to_qpairs import add_wv_to_qpairs
from keras.preprocessing.sequence import pad_sequences

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/data_0521/intermediate/'
KN_PAIRS_SAMPLED_TRAIN_CSV = os.path.join(KN_PAIRS_SAMPLED_PATH, 'kn_pairs_sampled_train.csv')
KN_PAIRS_SAMPLED_TEST_CSV = os.path.join(KN_PAIRS_SAMPLED_PATH, 'kn_pairs_sampled_test.csv')

In [5]:
kn_pairs_sampled_train = pd.read_csv(KN_PAIRS_SAMPLED_TRAIN_CSV)
kn_pairs_sampled_train

,label,q1,q2,q1_q2_intersect
0,1,Q497036,Q562048,0
1,1,Q419417,Q488916,0
2,1,Q448014,Q677058,0
3,1,Q569080,Q280483,0
4,1,Q180301,Q595401,0
5,1,Q629986,Q054411,0
6,1,Q312054,Q448822,0
7,1,Q628124,Q508358,0
8,1,Q616403,Q499862,0
9,1,Q355287,Q671234,0


In [6]:
kn_pairs_sampled_train.columns = ['label', 'qid1', 'qid2', 'q1_q2_intersect']
kn_pairs_sampled_train

,label,qid1,qid2,q1_q2_intersect
0,1,Q497036,Q562048,0
1,1,Q419417,Q488916,0
2,1,Q448014,Q677058,0
3,1,Q569080,Q280483,0
4,1,Q180301,Q595401,0
5,1,Q629986,Q054411,0
6,1,Q312054,Q448822,0
7,1,Q628124,Q508358,0
8,1,Q616403,Q499862,0
9,1,Q355287,Q671234,0


In [7]:
kn_pairs_sampled_test = pd.read_csv(KN_PAIRS_SAMPLED_TEST_CSV)
kn_pairs_sampled_test

,label,q1,q2,q1_q2_intersect
0,1,Q463697,Q426562,0
1,1,Q468437,Q035697,0
2,1,Q163402,Q704181,0
3,1,Q653825,Q118880,0
4,1,Q417981,Q688532,0
5,1,Q466377,Q349058,0
6,1,Q210382,Q381299,0
7,1,Q550846,Q648488,0
8,1,Q159822,Q382359,0
9,1,Q706165,Q715688,0


In [8]:
kn_pairs_sampled_test.columns = ['label', 'qid1', 'qid2', 'q1_q2_intersect']
kn_pairs_sampled_test

,label,qid1,qid2,q1_q2_intersect
0,1,Q463697,Q426562,0
1,1,Q468437,Q035697,0
2,1,Q163402,Q704181,0
3,1,Q653825,Q118880,0
4,1,Q417981,Q688532,0
5,1,Q466377,Q349058,0
6,1,Q210382,Q381299,0
7,1,Q550846,Q648488,0
8,1,Q159822,Q382359,0
9,1,Q706165,Q715688,0


In [9]:
with open(QUESTION_PKL, 'rb') as f:
    qs = pickle.load(f)

qs

,qid,words,chars
0,Q000000,W05733 W05284 W09158 W14968 W07863,L1128 L1861 L2218 L1796 L1055 L0847 L2927
1,Q000001,W17378 W17534 W03249 W01490 W18802,L2214 L1980 L0156 L1554 L2218 L1861 L3019 L010...
2,Q000002,W17378 W08158 W20171 W11246 W14759,L2214 L2350 L2568 L1969 L2168 L0694 L3012 L256...
3,Q000003,W11385 W14103 W02556 W13157 W09749,L0762 L2172 L1971 L2034 L2039 L2271 L1346 L223...
4,Q000004,W17508 W18238 W02952 W18103,L0018 L2321 L1346 L3019 L0104 L0902 L0354 L0362
5,Q000005,W17378 W12908 W19355 W08410 W11922 W14496 W20487,L2214 L1331 L0016 L1132 L2237 L1900 L0578 L195...
6,Q000006,W04346 W17378 W06112 W05733 W18238 W05284 W118...,L2323 L1526 L2214 L0358 L1849 L1128 L1861 L301...
7,Q000007,W16696 W02556 W16564 W03914 W18103,L0366 L2028 L2034 L2039 L0605 L1824 L2292 L015...
8,Q000008,W17378 W11385 W18238 W13157 W12244 W16319 W18448,L2214 L0762 L2172 L3019 L0104 L2271 L1346 L014...
9,Q000009,W14113 W16319 W18399 W03472 W11399 W07863 W18103,L0947 L2323 L2582 L0143 L2600 L2272 L1872 L123...


In [10]:
with open(WEM_PKL, 'rb') as f:
    wmap = pickle.load(f)

w2i = wmap['word2index']
w2i

{'W15017': 7122,
 'W00197': 4371,
 'W09109': 17658,
 'W10698': 1566,
 'W17244': 14715,
 'W05584': 1198,
 'W01381': 6426,
 'W12691': 682,
 'W16473': 13591,
 'W09931': 18426,
 'W19729': 17505,
 'W03875': 12915,
 'W13845': 8660,
 'W08501': 1582,
 'W12198': 8790,
 'W13518': 13776,
 'W10595': 15626,
 'W15551': 1029,
 'W10725': 12880,
 'W10799': 4073,
 'W19906': 14107,
 'W00270': 15230,
 'W09890': 148,
 'W08833': 5121,
 'W00226': 180,
 'W15519': 2545,
 'W07530': 14942,
 'W17824': 19196,
 'W09887': 2242,
 'W18205': 17861,
 'W09184': 3756,
 'W00085': 6323,
 'W02773': 6514,
 'W20660': 20633,
 'W13562': 2470,
 'W10048': 7782,
 'W17849': 9802,
 'W00922': 7119,
 'W00893': 3517,
 'W03725': 10732,
 'W01660': 19114,
 'W13584': 1640,
 'W00093': 11115,
 'W07124': 10424,
 'W09199': 16222,
 'W13387': 12405,
 'W20262': 6637,
 'W07108': 1748,
 'W19260': 3773,
 'W06942': 9330,
 'W18280': 6681,
 'W12672': 15254,
 'W07009': 17852,
 'W02845': 19217,
 'W10463': 4614,
 'W02307': 6619,
 'W10831': 4828,
 'W12958':

In [11]:
with open(CEM_PKL, 'rb') as f:
    cmap = pickle.load(f)

c2i = cmap['char2index']
c2i

{'L1907': 442,
 'L0651': 167,
 'L2389': 2879,
 'L0660': 2444,
 'L0594': 2623,
 'L0705': 242,
 'L2428': 811,
 'L0985': 1136,
 'L0961': 1525,
 'L3038': 1030,
 'L0990': 1728,
 'L2357': 2254,
 'L1291': 1974,
 'L2044': 1203,
 'L0736': 2158,
 'L1479': 2056,
 'L1974': 2389,
 'L2306': 549,
 'L0820': 339,
 'L0359': 1880,
 'L1398': 166,
 'L2196': 2860,
 'L2712': 1438,
 'L0033': 219,
 'L0234': 1691,
 'L2639': 2089,
 'L2174': 1553,
 'L0446': 263,
 'L0037': 2701,
 'L1842': 1918,
 'L2828': 2146,
 'L2418': 1848,
 'L1187': 1834,
 'L2275': 2502,
 'L1462': 1972,
 'L0614': 2180,
 'L0420': 2083,
 'L2590': 641,
 'L2993': 3045,
 'L0567': 646,
 'L1671': 981,
 'L1772': 2107,
 'L2129': 2641,
 'L2734': 625,
 'L1881': 2150,
 'L2300': 1051,
 'L1568': 2988,
 'L0372': 2644,
 'L1101': 977,
 'L0188': 2535,
 'L1042': 1062,
 'L0817': 955,
 'L1914': 894,
 'L1507': 492,
 'L1360': 2085,
 'L1628': 2822,
 'L0921': 2594,
 'L2160': 405,
 'L1920': 1492,
 'L0117': 448,
 'L1210': 2699,
 'L2510': 347,
 'L2935': 2512,
 'L0559': 19

In [12]:
kn_pairs_sampled_wids_cids_train = add_wv_to_qpairs(kn_pairs_sampled_train, qs, w2i, c2i)
kn_pairs_sampled_wids_cids_train

,label,qid1,qid2,q1_q2_intersect,words1,chars1,words2,chars2
0,1,Q497036,Q562048,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,Q572006,Q562048,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1,Q255826,Q562048,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,Q444476,Q562048,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0,Q555988,Q562048,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,1,Q652941,Q562048,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,0,Q091525,Q562048,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,0,Q652480,Q562048,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,1,Q049878,Q562048,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,0,Q453132,Q562048,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
kn_pairs_sampled_wids_cids_test = add_wv_to_qpairs(kn_pairs_sampled_test, qs, w2i, c2i)
kn_pairs_sampled_wids_cids_test

,label,qid1,qid2,q1_q2_intersect,words1,chars1,words2,chars2
0,1,Q463697,Q426562,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,Q405900,Q426562,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1,Q160551,Q426562,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0,Q138477,Q426562,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0,Q077693,Q426562,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,0,Q650363,Q426562,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,0,Q148251,Q426562,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,1,Q463697,Q199547,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,1,Q064245,Q199547,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,0,Q215247,Q199547,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
# 保存
KN_PAIRS_SAMPLED_WIDS_CIDS_TRAIN_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'kn_pairs_sampled_wids_cids_train.pkl')
KN_PAIRS_SAMPLED_WIDS_CIDS_TEST_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'kn_pairs_sampled_wids_cids_test.pkl')
with open(KN_PAIRS_SAMPLED_WIDS_CIDS_TRAIN_PKL, 'wb') as f:
    pickle.dump(kn_pairs_sampled_wids_cids_train, f)

with open(KN_PAIRS_SAMPLED_WIDS_CIDS_TEST_PKL, 'wb') as f:
    pickle.dump(kn_pairs_sampled_wids_cids_test, f)
